# Visualising Temporal Analysis Interactively

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/ai4er-cdt/gtc-biodiversity/main?filepath=notebooks%2F4-demo-geographviewer-chernobyl.ipynb)

This tutorial shows how to create and visualise a timeline of GeoGraphs.

---

## 1. Setup and Loading package

In [1]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

Let us start by installing all relevant dependencies

In [2]:
import ipyleaflet
import pandas as pd
import geopandas as gpd
import rioxarray as rxr

import geograph
from geograph import geotimeline
from geograph.visualisation import geoviewer
from geograph.constants import UTM35N
from geograph.demo.binder_constants import DATA_DIR, ROIS, ESA_CCI_LEGEND_LINK
from geograph.metrics import LANDSCAPE_METRICS_DICT, COMPONENT_METRICS_DICT

/home/users/svm/Code/gtc-biodiversity/env/lib/python3.8/site-packages/geopandas/_compat.py:84: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


---

## 2. Loading Data

Next, we will load the data for the Chernobyl region. For this example we will use land cover maps from the [ESA CCI land cover](http://www.esa-landcover-cci.org/) dataset. Specifically, we will look at the years 2013 and 2014 for the [Chernobyl exclusion zone](https://en.wikipedia.org/wiki/Chernobyl_Exclusion_Zone). All data comes pre-installed on the binder in the `DATA_DIR` that we imported from the `binder_constants`. If you are following this demo on your local machine, you can download the data with this link.

In [3]:
# Parse geotif landcover data
chernobyl_path = lambda year: DATA_DIR / "chernobyl" / "esa_cci" / f"esa_cci_{year}_chernobyl.tif" 

# Parse ROIS
rois = gpd.read_file(ROIS)
# Load the shape of the chernobyl exclusion zone
cez = rois[rois["name"] == "Chernobyl Exclusion Zone"]

In [4]:
def clip_and_reproject(xrdata, clip_geometry=None, to_crs=UTM35N, x_res=300, y_res=300):
    
    if clip_geometry is not None:
        clipped_data = xrdata.rio.clip(clip_geometry)
    else:
        clipped_data = xrdata
        
    if to_crs is not None:
        reprojected_data = clipped_data.rio.reproject(to_crs, resolution=(x_res, y_res))
    else:
        reprojected_data = clipped_data
    
    return reprojected_data

In [5]:
# Loading ESA CCI land cover raster data
years = list(range(2013,2015))
cez_rasters = {year: clip_and_reproject(rxr.open_rasterio(chernobyl_path(year)), clip_geometry=cez.geometry) 
              for year in years
             }

In [6]:
# Loading ESA CCI land cover legend to translate land cover labels to classes
esa_cci_legend = pd.read_csv(ESA_CCI_LEGEND_LINK, delimiter=";", index_col=0)
print(f"There are {len(esa_cci_legend)} classes.")

class_value_to_label = {class_val: row.LCCOwnLabel for class_val, row in esa_cci_legend.iterrows()}

There are 38 classes.


---

## 3. Creating `GeoGraph`

In [7]:
# Polygonising raster and transforming into graph
cez_graphs = {}
for year, raster in cez_rasters.items():
    print(f"Analysing year {year}")
    # Load geograph from the raster data (construction takes ~10s)
    cez_graphs[year] = geograph.GeoGraph(data=raster.data, 
                                transform=raster.rio.transform(), 
                                crs=UTM35N, 
                                mask = raster.data > 0,
                                connectivity=8)
    # Map the ESA CCI land cover class value (int) to the plain text label
    #  to have the plain text labels available in the interactive viewer.
    cez_graphs[year].df.class_label = cez_graphs[year].df.class_label.apply(lambda x: class_value_to_label[x])

Analysing year 2013


Identifying nodes: 100%|██████████| 2923/2923 [00:05<00:00, 556.09it/s]
Step 1 of 2: Creating nodes and finding neighbours: 100%|██████████| 2003/2003 [00:10<00:00, 191.28it/s]
Step 2 of 2: Adding edges: 100%|██████████| 2003/2003 [00:00<00:00, 69617.74it/s]


Graph successfully loaded with 2003 nodes and 5140 edges.
Analysing year 2014


Identifying nodes: 100%|██████████| 2915/2915 [00:05<00:00, 541.60it/s]
Step 1 of 2: Creating nodes and finding neighbours: 100%|██████████| 1999/1999 [00:11<00:00, 179.66it/s]
Step 2 of 2: Adding edges: 100%|██████████| 1999/1999 [00:00<00:00, 60868.21it/s]

Graph successfully loaded with 1999 nodes and 5117 edges.


---

## 4. Creating `Timeline` and identifying nodes:

In [8]:
cez_timeline = geotimeline.GeoGraphTimeline(cez_graphs)
# Perform node identification
cez_timeline.timestack()
# Classify node dynamics for the year 2014
cez_timeline.calculate_node_dynamics(2014);

Identifying nodes: 100%|██████████| 2003/2003 [00:03<00:00, 514.92it/s]


---

## 5. Inspect in interactive viewer

In [9]:
# Choose metrics to display:
metric_list = list(LANDSCAPE_METRICS_DICT.keys()) + list(COMPONENT_METRICS_DICT.keys())
# Build up the viewer
viewer = geoviewer.GeoGraphViewer(small_screen=True, metric_list=metric_list)
viewer.add_layer(ipyleaflet.basemaps.Esri.WorldImagery)
viewer.add_graph(cez_timeline[2014], name='Chernobyl data', with_components=False)
viewer.enable_graph_controls()

Calculating component polygons...


Constructing graph: 100%|██████████| 1/1 [00:00<00:00, 2499.59it/s]


              than ~100 components.


Constructing graph: 100%|██████████| 1/1 [00:00<00:00, 3179.91it/s]
Calculating edge weights: 0it [00:00, ?it/s]


In [10]:
# Display the interactive map
viewer

GeoGraphViewer(center=[51.389167, 30.099444], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom…

> Note: an interactive viewer will show up here.